# Train MNIST Classifier and Export for TF Lite Micro

This notebook describes how to create and train a simple MNIST classification model and then convert the model to a form ready to be compiled into a Tensorflow lite micro project. In this example all these steps are performed by a single python script.

The classification model is trained for 1000 steps to reduce execution time, since the main purpose of this notebook is to demonstrate the export process.

First we import neccessary packages and modules and load the MNIST dataset.

In [1]:
import os
import sys
import numpy as np

import tensorflow as tf
import fetch_mnist
import layers
import flatbuffer_2_tfl_micro as save_tflm
import progress_bar as pb

print("Getting training data")
mnist = fetch_mnist.MNISTDataset()
print("Done")

Getting training data
Done


### Building the Model

We build a simple classification model comprised of three fully connected layers.

In [2]:
sess = tf.compat.v1.InteractiveSession()
tf.compat.v1.disable_eager_execution()

with tf.name_scope('input'):
    x = tf.compat.v1.placeholder(tf.float32, [None, 28, 28], name='x-input')
    y_ = tf.compat.v1.placeholder(tf.int64, [None], name='y-input')

x_flat = tf.reshape(x, [-1, 784], name='x_flat')
hidden1 = layers.nn_layer(x_flat, 784, 25, 'layer1', act=tf.nn.relu)
hidden2 = layers.nn_layer(hidden1, 25, 25, 'layer2', act=tf.nn.relu)
y = layers.nn_layer(hidden2, 25, 10, 'layer3', act=tf.identity)
classifcation = tf.compat.v1.argmax(y,
                                    name='classification',
                                    axis=1,
                                    output_type=tf.dtypes.int32)
print('Model Built Okay.')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model Built Okay.


### Training the Model

The model is trained for 1000 iterations, using the cross entropy loss function.

In [3]:
[train_step, accuracy] = layers.cross_entropy_training(y, y_)
tf.compat.v1.global_variables_initializer().run()
training_steps = 1000
def feed_dict(train_d):
    if train_d:
        xs, ys = mnist.get_batch(100)
    else:
        xs, ys = mnist.x_test, mnist.y_test
    return {x: xs, y_: ys}

acc = 0
for i in range(training_steps):
    if i % 10 == 0:  # Record summaries and test-set accuracy
        acc = sess.run([accuracy], feed_dict=feed_dict(False))
        pb.update_progress_bar(i / float(training_steps),
                             pre_msg='Training MNIST Classifier',
                             post_msg='Accuracy is %s' % acc, size=40)
    else:  # Record train set summaries, and train
        sess.run([train_step], feed_dict=feed_dict(True))
pb.update_progress_bar(1.0,
                       pre_msg='Training MNIST Classifier',
                       post_msg='Accuracy is %s' % acc, size=40,
                       c_return='\n')
print('Training Complete.')

Training MNIST Classifier [#######################################=] 100.00% Accuracy is [0.8941]
Training Complete.


### Exporting the Model for Tensorflow lite micro

Now our MNIST classifier has been built and trained we're ready to export the model so that it can be built into the Tensorflow lite micro project and read by the micro-interpreter. First the model is converted into a Tensorflow lite flatbuffer which can be used by the micro-interpreter. However since the micro-controllers which will run the model do not have file systems, this flatbuffer needs to be compiled into the binary. So the flatbuffer is converted into a **.cc** and **.h** file pair, so it can be compiled along with the rest of the project.
This is done using the **write_tf_lite_micro_model()** helper function provided with this example.

In [6]:
converter = tf.compat.v1.lite.TFLiteConverter.from_session(sess, [x], [classifcation])

save_tflm.write_tf_lite_micro_model(converter.convert(),
                                    base_file_name="mnist_model",
                                    data_variable_name="mnist_dense_model_tflite",
                                    header_comment="Example MNIST classification model,"
                                                   "for use with TFlite Micro")
print("Complete")

INFO:tensorflow:Froze 6 variables.
INFO:tensorflow:Converted 6 variables to const ops.
Complete


When this script has finished executing two new files **mnist_model.cc** and **mnist_model.h** have been generated, ready to be compiled into the MNIST Tensorflow lite micro example. 